In [0]:
!pip install pytorch_lightning 

In [0]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer
import torch.nn.functional as F

from PIL import Image
import glob

# tensorboard
%load_ext tensorboard

In [0]:
# read imgs
fold_path = './imgs/'
imgs = []
for imgs_path in glob.glob(fold_path + '*'):
    imgs.append(glob.glob(imgs_path + '/*'))

In [0]:
from torchvision.models import resnet18

In [0]:
# ResNet download
resnet = resnet18(pretrained=True)

In [0]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [0]:
labels = []
img_datas = torch.tensor([])

for i,imgs_arr in enumerate(imgs):
    
    for img in imgs_arr:
        im = Image.open(img)
        tensor_img = transform(im.convert('RGB'))

        # add batch size
        tensor_img = tensor_img.unsqueeze(0)
        # label
        labels.append(i)

        img_datas = torch.cat([img_datas, tensor_img],dim=0)

In [0]:
# torch int64
labels = torch.tensor(labels, dtype=torch.int64)
img_datas.dtype, labels.dtype

(torch.float32, torch.int64)

In [0]:
datasets = torch.utils.data.TensorDataset(img_datas, labels)

n_train = int(len(datasets) * 0.85)
n_val = len(datasets) - n_train

# fixed seed
torch.manual_seed(0)

# divide datasets
train,val = torch.utils.data.random_split(datasets,[n_train,n_val])

In [0]:
class TrainNet(pl.LightningModule):

    @pl.data_loader
    def train_dataloader(self):
        return torch.utils.data.DataLoader(train, self.batch_size,shuffle=True)
    
    def training_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        tensorboard_logs = {'train_loss': loss}
        results = {'loss': loss, 'log': tensorboard_logs}
        return results

class ValidationNet(pl.LightningModule):

    @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'avg_acc': avg_acc}
        results =  {'val_loss': avg_loss, 'val_acc': avg_acc, 'log': tensorboard_logs}
        return results

In [0]:
class FineTuningNet(TrainNet, ValidationNet):

    def __init__(self, batch_size=128):
        super().__init__()
        self.batch_size = batch_size
        
        self.conv = resnet18(pretrained=True) 
        self.fc1 = nn.Linear(1000, 100)
        self.fc2 = nn.Linear(100, 3)

        for param in self.conv.parameters():
            param.requires_grad = False

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [0]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# train
fine_net = FineTuningNet()
trainer = Trainer(gpus=1, max_epochs=100)
trainer.fit(fine_net)

INFO:lightning:GPU available: True, used: True
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
INFO:lightning:
   | Name                       | Type              | Params
-------------------------------------------------------------
0  | conv                       | ResNet            | 11 M  
1  | conv.conv1                 | Conv2d            | 9 K   
2  | conv.bn1                   | BatchNorm2d       | 128   
3  | conv.relu                  | ReLU              | 0     
4  | conv.maxpool               | MaxPool2d         | 0     
5  | conv.layer1                | Sequential        | 147 K 
6  | conv.layer1.0              | BasicBlock        | 73 K  
7  | conv.layer1.0.conv1        | Conv2d            | 36

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: Displayed epoch numbers in the progress bar start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [0]:
# result
trainer.callback_metrics

{'avg_acc': tensor(0.9692, device='cuda:0'),
 'epoch': 99,
 'loss': tensor(0.0350, device='cuda:0'),
 'train_loss': tensor(0.0350, device='cuda:0'),
 'val_acc': tensor(0.9692, device='cuda:0'),
 'val_loss': tensor(0.1742, device='cuda:0')}

In [0]:
# tensorborad
%tensorboard --logdir ./lightning_logs/version_0

In [0]:
# predict
x = val[0][0].unsqueeze(0).cuda()
fine_net.eval()
fine_net.freeze()
F.softmax(fine_net(x)), val[0][1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


(tensor([[0.0445, 0.0247, 0.9308]], device='cuda:0'), tensor(0))

In [0]:
# save the model
torch.save(fine_net.state_dict(), 'model.pt')

In [0]:
#predict
fien_net_pred = FineTuningNet()

fien_net_pred.eval()
fien_net_pred.freeze()

# read weights
fien_net_pred.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>